# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115664e+02     1.536012e+00
 * time: 0.1279900074005127
     1     1.018830e+01     8.177933e-01
 * time: 0.9498951435089111
     2    -1.191186e+01     9.448034e-01
 * time: 1.0828790664672852
     3    -3.415417e+01     7.302661e-01
 * time: 1.2522361278533936
     4    -4.763873e+01     5.653385e-01
 * time: 1.4071099758148193
     5    -5.693761e+01     2.054103e-01
 * time: 1.5507681369781494
     6    -5.972339e+01     1.649561e-01
 * time: 1.6677460670471191
     7    -6.085713e+01     6.247863e-02
 * time: 1.7788469791412354
     8    -6.135047e+01     4.257051e-02
 * time: 1.897355079650879
     9    -6.162743e+01     3.729123e-02
 * time: 2.00740909576416
    10    -6.181054e+01     3.260977e-02
 * time: 2.123703956604004
    11    -6.196758e+01     2.062483e-02
 * time: 2.2329111099243164
    12    -6.206152e+01     1.728121e-02
 * time: 2.3485920429229736
    13    -6.210215e+01     1.363844e-02
 * time: 2.45877909660339

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671052
    AtomicLocal         -18.8557599
    AtomicNonlocal      14.8522599
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485346 
    Xc                  -19.3336811

    total               -62.261666447040
